In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms, datasets, models
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json, random

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 2025
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
# Normalization constants

args = {
    "data":      "TestDataSet",
    "labels":    "TestDataSet/labels_list.json", 
    "batch":     128,
    "workers":   8,
    "rand_start": True
}

# ImageNet normalization
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
std_tensor = torch.tensor(std, device=device, dtype=torch.float32).view(3,1,1)
normalize = transforms.Normalize(mean.tolist(), std.tolist())

min_val  = torch.tensor(((0 - mean) / std).reshape(3,1,1), device=device)
max_val  = torch.tensor(((1 - mean) / std).reshape(3,1,1), device=device)

In [2]:
transform = transforms.Compose([transforms.ToTensor(), normalize])
dataset   = datasets.ImageFolder(root=args["data"], transform=transform)
loader    = DataLoader(dataset, batch_size=args["batch"], shuffle=False, num_workers=args["workers"])

with open(args["labels"]) as f:
    idx2true = {i: int(e.split(":",1)[0]) for i, e in enumerate(json.load(f))}

# ------------------- Load Pretrained Model -------------------
model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1).to(device)
model.eval(),
# ------------------- Evaluation Function -------------------
@torch.no_grad()
def evaluate(model, loader, idx2true, device, tag=""):
    top1 = top5 = tot = 0
    for x, labs in loader:
        x = x.to(device, non_blocking=True)
        labs = torch.tensor([idx2true[int(l)] for l in labs], device=device)
        logits = model(x)
        top1 += (logits.argmax(1) == labs).sum().item()
        top5 += (logits.topk(5, 1)[1] == labs[:, None]).any(1).sum().item()
        tot  += labs.size(0)
    print(f"{tag:<8}Top‑1 {top1/tot*100:6.2f}%   Top‑5 {top5/tot*100:6.2f}%")
    return top1 / tot, top5 / tot

In [3]:
# ------------------- Evaluate Clean Accuracy -------------------
evaluate(model, loader, idx2true, device, "Clean")

Clean   Top‑1  76.00%   Top‑5  94.20%


(0.76, 0.942)

In [4]:
def fgsm_batch(model, imgs, true_lbls, eps_norm, min_val, max_val):
    imgs.requires_grad_(True)
    logits = model(imgs)
    F.cross_entropy(logits, true_lbls).backward()
    adv = imgs + eps_norm * imgs.grad.data.sign()
    adv = torch.max(torch.min(adv, max_val), min_val)
    return adv.detach()

In [5]:
def pgd_attack(model, x, y, eps_n, alpha_n, steps, min_v, max_v, rand_start):
    if rand_start:
        x_adv = x + torch.empty_like(x).uniform_(-1, 1) * eps_n
        x_adv = torch.clamp(x_adv, min_v, max_v)
    else:
        x_adv = x.clone()

    for _ in range(steps):
        x_adv = x_adv.detach().requires_grad_(True)
        model.zero_grad(set_to_none=True)
        F.cross_entropy(model(x_adv), y).backward()
        grad_sign = x_adv.grad.sign()
        x_adv = x_adv + alpha_n * grad_sign
        x_adv = torch.clamp(x_adv, x - eps_n, x + eps_n)
        x_adv = torch.clamp(x_adv, min_v, max_v)
    return x_adv.detach()

def targeted_pgd(model, x, target, eps_n, alpha_n, steps, min_v, max_v):
    x_adv = x.clone()
    for _ in range(steps):
        x_adv = x_adv.detach().requires_grad_(True)
        model.zero_grad(set_to_none=True)
        loss = F.cross_entropy(model(x_adv), target)
        loss.backward()
        grad = x_adv.grad.sign()
        x_adv = x_adv - alpha_n * grad
        x_adv = torch.clamp(x_adv, x - eps_n, x + eps_n)
        x_adv = torch.clamp(x_adv, min_v, max_v)
    return x_adv.detach()

In [6]:
eps   = 0.02
steps = 5

args = {
    "data":      "TestDataSet",
    "labels":    "TestDataSet/labels_list.json",
    "eps":       eps,
    "steps":     steps,
    "alpha":     eps / steps,   
    "batch":     128,
    "workers":   8,
    "rand_start": True,
    "out":       "adv_set_pgd.pt"
}

eps_n   = torch.tensor(args["eps"]   / std, device=device, dtype=torch.float32).view(3,1,1)
alpha_n = torch.tensor(args["alpha"] / std, device=device, dtype=torch.float32).view(3,1,1)
# eps_n  = eps
# alpha_n = args["alpha"]
# Craft adversarial examples
adv_batches, lab_batches = [], []
for imgs, labs in loader:
    imgs = imgs.to(device, non_blocking=True)
    true = torch.tensor([idx2true[int(l)] for l in labs], device=device)
    adv = fgsm_batch(model, imgs, true, eps_n, min_val, max_val)

    # Check ε constraint in raw pixels
    raw_delta = (adv - imgs).abs() * torch.tensor(std, device=device).view(3,1,1)
    assert raw_delta.max().item() <= eps + 1e-6

    adv_batches.append(adv)
    lab_batches.append(labs.to(device))

adv_tensor = torch.cat(adv_batches)
lab_tensor = torch.cat(lab_batches)


In [7]:
# Evaluate on adversarial set
adv_loader = DataLoader(TensorDataset(adv_tensor, lab_tensor), batch_size=args["batch"])
evaluate(model, adv_loader, idx2true, device, "FGSM")

FGSM    Top‑1   3.60%   Top‑5  21.00%


(0.036, 0.21)

In [8]:
# ------------------- PGD Attack Loop -------------------
adv_batches, lab_batches = [], []
for x, labs in loader:
    x = x.to(device, non_blocking=True)
    y = torch.tensor([idx2true[int(l)] for l in labs], device=device)

    adv = pgd_attack(model, x, y, eps_n, alpha_n,
                     args["steps"], min_val, max_val, args["rand_start"])
    
    raw_delta = ((adv - x).abs() * std_tensor).max().item()
    assert raw_delta <= args["eps"] + 1e-6, f"ε-constraint {raw_delta:.6f} > ε"

    adv_batches.append(adv)
    lab_batches.append(labs.to(device))

adv_tensor = torch.cat(adv_batches)
lab_tensor = torch.cat(lab_batches)

# ------------------- Evaluate PGD Accuracy -------------------
pgd_loader = DataLoader(TensorDataset(adv_tensor, lab_tensor),
                        batch_size=args["batch"])
evaluate(model, pgd_loader, idx2true, device, "PGD")

PGD     Top‑1   0.00%   Top‑5   4.40%


(0.0, 0.044)

In [9]:
# ------------------- Parameters -------------------
args = {
    "data": "TestDataSet",
    "labels": "TestDataSet/labels_list.json",
    "eps": 0.02,
    "steps": 5,           # 比 FGSM 更强
    "alpha": 0.004,
    "batch": 64,
    "workers": 4,
    "target_class": 0,     # 所有图像强制扰动成这个类 (tench)
    "out": "adv_set_targeted.pt"
}
# ------------------- Targeted Attack Loop -------------------
adv_batches, lab_batches = [], []
for x, labs in loader:
    x = x.to(device)
    target = torch.full_like(labs, fill_value=args["target_class"], device=device)

    adv = targeted_pgd(model, x, target, eps_n, alpha_n,
                       args["steps"], min_val, max_val)

    # Ensure constraint
    raw_delta = ((adv - x).abs() * std_tensor).max().item()
    assert raw_delta <= args["eps"] + 1e-6, f"ε-constraint {raw_delta:.6f} > ε"

    adv_batches.append(adv)
    lab_batches.append(labs.to(device))

adv_tensor = torch.cat(adv_batches)
lab_tensor = torch.cat(lab_batches)

# ------------------- Evaluate Targeted PGD Accuracy -------------------
adv_loader = DataLoader(TensorDataset(adv_tensor, lab_tensor), batch_size=args["batch"])
evaluate(model, adv_loader, idx2true, device, "Targeted")

TargetedTop‑1   0.60%   Top‑5   6.40%


(0.006, 0.064)

### 🔁 PGD Evaluation Loop with Varying Steps

We will evaluate the effect of PGD and Targeted PGD attacks under varying step sizes (5, 10, 15). For each configuration:

- Clean accuracy is computed.
- FGSM attack is compared.
- PGD and targeted PGD are performed for each `steps` setting.
- Accuracy results are printed.
- Five sample images are saved for comparison across attacks.


In [10]:
from torchvision.utils import save_image
import os

# Create directory for saving results
os.makedirs("adv_examples", exist_ok=True)

step_list = [5, 10, 15]
fgsm_eps = 0.02
results = {}

# Randomly select 5 images
sample_images = []
sample_labels = []
for i, (img, lab) in enumerate(loader):
    for j in range(len(img)):
        if len(sample_images) < 5:
            sample_images.append(img[j])
            sample_labels.append(lab[j])
        else:
            break
    if len(sample_images) >= 5:
        break

sample_images = torch.stack(sample_images).to(device)
sample_labels = torch.tensor(sample_labels).to(device)
sample_true = torch.tensor([idx2true[int(l)] for l in sample_labels], device=device)

# Save clean versions
save_image(sample_images, "adv_examples/clean.png")

# FGSM attack
sample_images.requires_grad_()
out = model(sample_images)
loss = F.cross_entropy(out, sample_true)
model.zero_grad()
loss.backward()
grad = sample_images.grad.data
adv_fgsm = torch.clamp(sample_images + fgsm_eps / std_tensor * grad.sign(), min_val, max_val).detach()
save_image(adv_fgsm, "adv_examples/fgsm.png")
results["FGSM"] = evaluate(model, DataLoader(TensorDataset(adv_fgsm, sample_labels), batch_size=5),
                           idx2true, device, tag="FGSM")

# Loop over PGD steps
for steps in step_list:
    alpha = fgsm_eps / steps
    alpha_n = torch.tensor(alpha / std, device=device).view(3,1,1)
    eps_n = torch.tensor(fgsm_eps / std, device=device).view(3,1,1)

    # Untargeted PGD
    adv_pgd = pgd_attack(model, sample_images, sample_true, eps_n, alpha_n, steps,
                         min_val, max_val, rand_start=True)
    save_image(adv_pgd, f"adv_examples/pgd_{steps}.png")
    results[f"PGD-{steps}"] = evaluate(model,
        DataLoader(TensorDataset(adv_pgd, sample_labels), batch_size=5), idx2true, device, tag=f"PGD-{steps}")

    # Targeted PGD (choose arbitrary incorrect target label: (true+1)%1000)
    target_labels = (sample_true + 1) % 1000
    adv_target = targeted_pgd(model, sample_images, target_labels, eps_n, alpha_n, steps,
                              min_val, max_val, rand_start=False)
    save_image(adv_target, f"adv_examples/tpgd_{steps}.png")
    results[f"TPGD-{steps}"] = evaluate(model,
        DataLoader(TensorDataset(adv_target, sample_labels), batch_size=5), idx2true, device, tag=f"TPGD-{steps}")


FGSM    Top‑1  20.00%   Top‑5  20.00%
PGD-5   Top‑1   0.00%   Top‑5   0.00%


TypeError: targeted_pgd() got an unexpected keyword argument 'rand_start'

In [ ]:
# Summary of results
print("\n===== Accuracy Summary =====")
for k, v in results.items():
    print(f"{k:<10}: Top-1 {v[0]*100:.2f}%, Top-5 {v[1]*100:.2f}%")
